# Tutorial de Big Data
## Bienvenidos a la clase 9 

**Objetivo:**  
Que se familiaricen con las técnicas de Regularización de Ridge y Lasso

### Temario:
- Reglas de Slack
- Preguntas del TP
- Regularización con Ridge y Lasso
- Ejemplo párctico con base de hitters 

#### Modelos lineales regularizados con Ridge y Lasso

Exploraremos brevemente el conjunto de datos "Hitters" y usaremos la libreria de sklearn para ajustar modelos lineales regularizados por Lasso (son las siglas en ingles para: operador de selección y contracción mínima absoluta) y Ridge con el fin de predecir el salario de los jugadores de béisbol.

Esta es una adaptación a Python de R. Jordan Crouser de las páginas 251-254 de "Introduction to Statistical Learning with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani. 






#### 1. Leer el conjunto de datos y explorar la estructura de datos 

In [ ]:
# Importamos los paquetes necesarios
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


Les dejo la documentación para que puedan comparar:
- [LogisticRegression()](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) --> La técnica de regularización de L1 se llama Regresión de Lasso y la L2 se llama Regresión de Ridge
- [linear_model()](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model) --> [Lasso()](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso) y [Ridge()](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge)


![alt text](Lasso.png "Title")

![alt text](Ridge.png "Title")

![alt text](hitters.png)

In [ ]:
# read data set and drop missing values 
Hitters = pd.read_csv("Hitters.csv").dropna() 
Hitters.info()

In [ ]:
Hitters

#### 2. Preparar las X e Y que usaremos en el modelo

Aquí seleccionamos las variables que utilizaremos en nuestro modelo y transformamos a dummies las que son strings

In [ ]:
y = Hitters.Salary
# Creamos variables dummies para las variables string
dummies = pd.get_dummies(Hitters[['League', 'Division', 'NewLeague']])
dummies

In [ ]:
# Definimos las variables que incluiremos en el set de X.

# Eliminamos salarios (porque es nuestra y) y las columnas de strings
X_ = Hitters.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X.info()

#### 3. Ajuste un conjunto de modelos de lazo

Usamos la función Lasso () para realizar una regresión de lineal regularizada. La función Lasso () tiene un argumento alfa (es el λ pero con otro nombre) que se usa para ajustar el modelo.

Generaremos una matriz de valores alfa que van desde muy grandes a muy pequeños, esencialmente cubriendo la gama completa de escenarios desde el modelo nulo que contiene solo la intersección, hasta el ajuste de mínimos cuadrados.

Estandarizamos los datos y ajustamos los modelos Lasso para cada valor de alfa.

In [ ]:
alphas = 10**np.linspace(6,-2,50)*0.5
alphas

In [ ]:
lasso = Lasso(max_iter=10000, normalize=True)
coefs = []

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X, y)
    coefs.append(lasso.coef_)
    
np.shape(coefs)

#### 4. Plot Lasso parámetro de ajuste alfa

Ahora trazamos la relación entre alfa y los coeficientes, una línea para cada característica.

In [ ]:
ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')

En el lado derecho podemos ver el modelo nulo, que contiene solo la intersección. Esto se debe a una penalización muy alta. En el lado izquierdo casi no hay penalización. Observe que en la gráfica de coeficientes (dependiendo de la elección del parámetro de ajuste) algunos de los coeficientes son exactamente iguales a cero.

#### 5. Lazo con validación cruzada 

Ahora dividimos las muestras en un conjunto de entrenamiento y un conjunto de prueba para estimar el error de prueba. Realizamos una validación cruzada de 10 veces para elegir el mejor alfa, reajustar el modo, calcular el error de prueba asociado e imprimir los mejores coeficientes de modelos.

In [ ]:
# Usamos cross-validation para dividir la muestra en training y test sets
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

lassocv = LassoCV(alphas=None, cv=10, max_iter=100000, normalize=True)
lassocv.fit(X_train, y_train)
lasso.set_params(alpha=lassocv.alpha_)
print("Alpha=", lassocv.alpha_)
lasso.fit(X_train, y_train)
print("mse = ",mean_squared_error(y_test, lasso.predict(X_test)))
print("best model coefficients:")
pd.Series(lasso.coef_, index=X.columns)

Observamos que 13 de las 19 estimaciones de coeficientes son exactamente cero. Por lo tanto, Lasso tiene una ventaja sustancial sobre la regresión de Ridge en que realiza una selección de las variables del modelo. Para completar la imagen, necesitamos los resultados de validación cruzada correspondientes de la regresión de la cresta.

#### 6. Regresión con Ridge

In [ ]:
# Split data into training and test sets
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

Probemos elegir arbitrariamente el alpha usando $\lambda = 4$:

In [ ]:
ridge2 = Ridge(alpha = 4, normalize = True)
ridge2.fit(X_train, y_train)             
pred2 = ridge2.predict(X_test)           
print(pd.Series(ridge2.coef_, index = X.columns)) 
print(mean_squared_error(y_test, pred2))          

El ECM con alpha = 4 es 106216. 

Ahora probemos que pasa con un alpha muy grande, por ej. $10^{10}$:

In [ ]:
ridge3 = Ridge(alpha = 10**10, normalize = True)
ridge3.fit(X_train, y_train)             
pred3 = ridge3.predict(X_test)           
print(pd.Series(ridge3.coef_, index = X.columns)) 
print(mean_squared_error(y_test, pred3))          

Esta gran penalización reduce los coeficientes en un grado muy grande, esencialmente reduciéndose a un modelo que contiene solo el intersepto. Esta contracción excesiva hace que el modelo sea más sesgado, lo que resulta en un ECM más alto.

Bien, entonces ajustar un modelo de regresión de Ridge con alfa = 4 conduce a un MSE de prueba mucho más bajo que ajustar un modelo con solo una intersección. Ahora verificamos si existe algún beneficio al realizar la regresión de Ridge con alfa = 4 en lugar de simplemente realizar la regresión por mínimos cuadrados. (Recuerden que la MCO es simplemente una regresión de Ridge con alfa = 0).

In [ ]:
ridge2 = Ridge(alpha = 0, normalize = True)
ridge2.fit(X_train, y_train)             
pred = ridge2.predict(X_test)            
print(pd.Series(ridge2.coef_, index = X.columns)) 
print(mean_squared_error(y_test, pred))           

De hecho, el modelo con Ridge es mejor que el de mínimos cuadrados regulares al comparar los ECM.

Como era de esperar, ninguno de los coeficientes es exactamente cero: por que la regresión de Ridge no realiza  selección de variables!

Ahora, en lugar de elegir arbitrariamente alfa $ = 4 $, sería mejor usar la validación cruzada para elegir el parámetro de ajuste alfa. 

#### Ridge con validación cruzada 

Nuevamente, realizamos una validación cruzada con k=10 para elegir el mejor alfa, reajustar el modo, calcular el error de prueba asociado e imprimir los mejores coeficientes de modelos. Esta vez regularizando con Ridge

In [ ]:
ridgecv = RidgeCV(alphas=alphas, normalize=True)
ridgecv.fit(X_train, y_train)
print("Alpha=", ridgecv.alpha_)
ridge6 = Ridge(alpha=ridgecv.alpha_, normalize=True)
ridge6.fit(X_train, y_train)
print("mse = ",mean_squared_error(y_test, ridge6.predict(X_test)))
print("best model coefficients:")
pd.Series(ridge6.coef_, index=X.columns)

La performance de los modelos regularizados con Ridge y con Lasso con Cross-validation:

In [ ]:
print("mse ridge = ",mean_squared_error(y_test, ridge6.predict(X_test)))
print("mse lasso = ",mean_squared_error(y_test, lasso.predict(X_test)))

#### Resumen
Con alfa elegido por validación cruzada en este ejemplo, la prueba MSE del Lasso es un poco peor que la prueba MSE de regresión de Ridge. 

El lazo tiene una gran ventaja sobre la regresión de crestas, ya que produce modelos más simples e interpretables que involucran solo a un subconjunto de predictores.
